In [4]:
import aiohttp
import os
from pathlib import Path
import pandas as pd
import utils
import asyncio
import math

In [5]:
try:
    os.mkdir(utils.DATASETS_AMAZON_IMAGES_PATH)
except:
    pass

In [6]:
df = pd.read_csv(Path('datasets/amazon/books_data_reordered.csv'), usecols=['Title', 'image'])
df = df[df['image'].isna() == False]

In [7]:
async def load__and_save_img(arr, session):
    title, image_url = arr
    title_path = utils.convert_str_to_datasets_amazon_images_path(title)
    # is_file checks if file exists
    if title_path.is_file() == True:
        return
    async with session.get(image_url) as response:
        content = await response.read()
        print(content)
    # if response.status == 200:
    #     with open(title_path, 'wb') as file:
    #         file.write(content)
    # else:
    #     print(f'Status code : {response.status}')
    #     raise Exception()

In [8]:
async def save_images(start, count, session):
    _ = [await load__and_save_img(row, session) for row in df.values[start:start + count]]

In [ ]:
async def run_tasks():
    batch_size = 3
    end = math.ceil(df.shape[0] / batch_size)
    async with aiohttp.ClientSession() as session:
        tasks = [save_images(i * batch_size, batch_size, session) for i in range(5)]
        results = await asyncio.gather(*tasks)
        return results

await run_tasks()